# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. 

**Цель исследования** - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Вопросы, на которые необходимо дать ответ по итогу исследования:
* "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
* "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
* "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
* "Как разные цели кредита влияют на его возврат в срок?";

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Ход исследования**

 1. Обзор данных.
 2. Предобработка данных.
 3. Формулировка ответа на поставленный вопрос.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в переменную data
data = pd.read_csv('/datasets/data.csv')

In [3]:
# получение первых 10 строк таблицы data
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о данных в таблице data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Общие представления о таблице есть. Теперь сделаем обзор категориальных и числовых переменных, неоходимых для исследования.

In [5]:
# уникальные значения и количество их упоминаний в колонке children
display(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [6]:
# уникальные значения и количество их упоминаний в колонке family_status
display(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [7]:
# уникальные значения и количество их упоминаний в колонке income_type
display(data['income_type'].value_counts())


сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [8]:
# уникальные значения и количество их упоминаний в колонке 'purpose'
display(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [9]:
# уникальные значения и количество их упоминаний в колонке 'debt'
display(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64

In [10]:
# обзор числовых значений в колонке 'days_employed'
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

In [11]:
# обзор числовых значений в колонке 'total_income'
data['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

Итак, в таблице семь столбцов. 

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В названиях колонок нарушения стиля не наблюдается.

**Вывод**

В таблице представлена статистика о платёжеспособности клиентов банка. Каждая строка отражает персональную информацию каждого заемщика - семейное положение, образование, информация о трудовой деятельности и доходах, цель получения кредита.

Главная цель проекта - выяснить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.
 
Предварительно можно утверждать, что, данных достаточно для выяснения поставленного вопроса.
На данном этапе выявлены следующие замечания в таблице:
1. Встречаются пропуски в данных в колонках  `total_income` и `days_employed`.
2. В колонке 'children' возможны опечатки ( 76 строк со значениями 20 детей, что не похоже на действительность).
3. В колонке 'purpose' и 'education' встречаются дубликаты, которые необходимо будет обработать. 
4. В колонке "days_employed"  в данных  встречаться артефакты — значения, которые не отражают действительность.Это можно наблюдать с отрицательным количеством дней трудового стажа. Также, опираясь на максимальное значение, есть предположение, что не все данные в этой колонке обозначены в именно в днях.

Чтобы двигаться дальше, нужно  описать возможные причины появления проблем в данных и обработать их.

## Шаг 2. Предобработка данных

### Обработка пропусков

Сначала призведем подсчет пропущенных значений в таблице.

In [12]:
# подсчёт пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Пропущенные значения выявлены в двух столбцах - 'days_employed' и 'total_income'. В каждой строке 2174 пропущенных значения т.е. около 10 % пропущенных значений. 

Но прежде чем обрабатывать пропуски в колонке 'days_employed', необходимо исправить некоррекные  данные. 
Отрицательные значения возьмем по модулю. А для слишком больших значений ( которые предположительно выражены не в днях, а в часах), определим возможный порог по максимальному возрасту клиента.

In [13]:
# берем значения по модулю
data['days_employed'] = data['days_employed'].apply(abs)

In [14]:
# по максимальному значению возраста берем ориентровочный порог, для перевода значений в часах в дни
print("Максимальное значение возраста клиента",data['dob_years'].max(),",значит возьмем ориентировочный порог - ",(data['dob_years'].max()-15)*365)


Максимальное значение возраста клиента 75 ,значит возьмем ориентировочный порог -  21900


In [15]:
#осуществляем певевод значений по порогу и выведем результат изменений
data.loc[data['days_employed'] > 21900, 'days_employed'] = data.loc[data['days_employed'] >21900, 'days_employed'] / 24
data['days_employed'].describe()

count    19351.000000
mean      4641.641176
std       5355.964289
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.882441
max      18388.949901
Name: days_employed, dtype: float64

Замена некорректных значений выполена.
Выясним как распределены пропуски по категориям,чтобы понять какими значениями заполять пропуски.

In [16]:
# логический признак отсутсвия данных- распределение пропусков в доходах и стажах
data['missing_days_employed'] = data['days_employed'].isna()
data['missing_total_income'] = data['total_income'].isna()
display(data.groupby('income_type').agg({'missing_days_employed': "mean",'missing_total_income':'mean'}).sort_values(by='missing_days_employed',ascending=False))
display(data.groupby('family_status').agg({'missing_days_employed': "mean",'missing_total_income':'mean'}).sort_values(by='missing_days_employed',ascending=False))

,missing_days_employed,missing_total_income
income_type,,
предприниматель,0.500000,0.500000
пенсионер,0.107106,0.107106
госслужащий,0.100754,0.100754
компаньон,0.099902,0.099902
сотрудник,0.099379,0.099379
безработный,0.000000,0.000000
в декрете,0.000000,0.000000
студент,0.000000,0.000000


,missing_days_employed,missing_total_income
family_status,,
гражданский брак,0.105818,0.105818
Не женат / не замужем,0.102382,0.102382
женат / замужем,0.099919,0.099919
вдовец / вдова,0.098958,0.098958
в разводе,0.093724,0.093724


Проанализировав  долю пропусков в каждой категории можно сделать вывод, что пропуски не зависят от семейного положения и явно зависят от типа дохода. Во второй таблице в каждой категории доля составляет 0.09-0.1 (т.е. порядка 10%). То-есть пропуски равномерно распределены по всем категориям семейного положения. В первой же таблице видно, что равномерность нарушается.

Также категории с пропусками явно совпадают для обоих столбцов, поэтому принято решение заполнить пропуски в обоих столбцах медианой с группировкой по типам дохода.

In [17]:
# заполнение пропусков в столбце 'days_employed' с группировкой по типам дохода 
for income in data['income_type'].unique():
    median = data.loc[data['income_type']==income,'days_employed'].median()
    data.loc[(data['days_employed'].isna()) &( data['income_type'] == income),'days_employed'] = median
      

In [18]:
# заполнение пропусков в столбце 'total_income' с группировкой по типам дохода 
for income in data['income_type'].unique():
    median = data.loc[data['income_type'] == income,'total_income'].median()
    data.loc[(data['total_income'].isna()) &( data['income_type'] == income),'total_income'] = median

Замена значений выполнена. Проверяем не осталось ли в таблице пропусков.

In [19]:
# подсчет пропусков
data.isna().sum()

children                 0
days_employed            0
dob_years                0
education                0
education_id             0
family_status            0
family_status_id         0
gender                   0
income_type              0
debt                     0
total_income             0
purpose                  0
missing_days_employed    0
missing_total_income     0
dtype: int64

**Вывод**

Изначально в таблице были выявлены пропущенные значения в двух столбцах - 'days_employed' и 'total_income'. В каждой строке 2174 пропущенных значения т.е. около 10 % пропущенных значений.

В связи с тем, что одним из вопросов исследования является определение зависимости между уровнем дохода и возвратом кредита в срок, данные из столбцов с пропусками нам будут необходимы для дальнейшего анализа.  

Заполнение пропусков в столбце 'total_income' и 'days_employed' было осуществлено с группировкой по типам дохода.

Так как эти столбцы связаны с трудовой деятельностью, возможная причина появления пропусков связана с видом деятельности ( большинство пропусков было выявлено среди предпринимателей), с определенным социальным статусом (пенсионер,госслужащий), либо с отсутствием подтверждения данных о доходе.

### Замена типа данных

В колонках 'days_employed' данные представлены в числовом типе float64 (вещественное число).

Так как значения в столбце 'days_employed' выражены в днях, заменим вещественный тип данных на целочисленный.

In [20]:
# замена типа данных на integer (целое число) в столбце 'days_employed':
data['days_employed'] = data['days_employed'].astype('int')

In [21]:
#проверка замены типа данных
data.dtypes

children                   int64
days_employed              int64
dob_years                  int64
education                 object
education_id               int64
family_status             object
family_status_id           int64
gender                    object
income_type               object
debt                       int64
total_income             float64
purpose                   object
missing_days_employed       bool
missing_total_income        bool
dtype: object

**Вывод**

Замена вещественного типа данных на целочисленный в колонках 'days_employed' была призведена методом astype(),который универсален и используется для перевода любых данных в нужный тип значения. 

### Обработка дубликатов

In [22]:
#изначальное количество дубликатов в таблице
data.duplicated().sum()

54

Изначально в таблице выявлено 54 дубликата. Но при первоначальном обзоре данных было выявлено,что 
в колонке 'education' буквы отличаются регистром, что может стать причиной пропусков явных дубликатов в таблице.

Проверим эту теорию.Переведем значения столбца education в нижний регистр.

In [23]:
# уникальные значения в столбце 'education'
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [24]:
# уникальные значения в столбце 'education' после перевода в нижний регистр
data['education']=data['education'].str.lower()

In [25]:
# количество дубликатов после обработки
data.duplicated().sum()

71

Действительно, после перевода значений в нижний регистр в столбце education, дубликатов в общей таблице обнаружено больше. 

Избавимся от дубликатов методом drop_duplicates и обновим индексы.

In [26]:
#удаление дубликатов и обновление индексов 
data=data.drop_duplicates().reset_index(drop = True)

In [27]:
# проверка на наличие дубликатов
data.duplicated().sum()

0

**Вывод**

В таблице был выявлен 71 дубликат, которые были удалены методом drop_duplicates().

Возможные причины возникновения дубликатов — повторные представления, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации.

### Лемматизация

Для дальнейшего анализа в значениях столбца с целями получения кредита необходимо лемматизировать значения- т.е. привести все слова к их словарной форме (лемме). Для этого воспользуемся функцией лемматизации на русском языке — pymystem3.

In [28]:
# импортируем pymystem3 :
from pymystem3 import Mystem
m = Mystem() 

In [29]:
# получение уникальных значений из столбца purpose и сохранение в переменную "purpose_str" 
purpose_str = data['purpose'].unique()
# склеивание списка значений purpose_str в одну строку методом join для дальнейшей лемматизации
purpose = " ".join(purpose_str)

In [30]:
# лемматизация 
lemmas = m.lemmatize(purpose)
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

In [31]:
# подсчёт числа упоминаний каждого из слов в тексте
from collections import Counter
print(Counter(lemmas)) 

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Выделяем из списка самые частые существительные : 'недвижимость' , 'жилье', 'автомобиль', 'образование','свадьба'. Берем их за основу при распределении на категории и создаем функцию для распределения.

In [32]:
# создадим функцию для проверки каждого значения столбца purpose и присвоение соответсвующей категории
def categorize(purpose):
    purpose_lemmas = m.lemmatize(purpose)
    if 'недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas:
        return 'недвижимость'
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if 'образование' in purpose_lemmas:
        return 'образование'
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'
    return 'прочее'



In [33]:
#создание нового столбца для обозначения категории с помощью метода apply и ранее созданной функции 
data['category'] = data['purpose'].apply(categorize)
#выведем 10 первых значений на экран для наглядности выполнения функции
(data.loc[:,['category','purpose']]).head(10)

,category,purpose
0,недвижимость,покупка жилья
1,автомобиль,приобретение автомобиля
2,недвижимость,покупка жилья
3,образование,дополнительное образование
4,свадьба,сыграть свадьбу
5,недвижимость,покупка жилья
6,недвижимость,операции с жильем
7,образование,образование
8,свадьба,на проведение свадьбы
9,недвижимость,покупка жилья для семьи


**Вывод**

В связи с тем, что в столбце purpose, были обнаружены значения, записанные в разных формах, но которые по факту имели одинаковый смысл, возникла необходимость прибегнуть к лемматизации. 

После обработки данных столбца мы добавили новый столбец "category" с распределением по самым популярным причинам получения займа -  'недвижимость' , 'жилье', 'автомобиль', 'образование','свадьба', с которым будет легче работать при ответе на вопрос о зависимости причины кридита и невозврата его в срок.

### Категоризация данных

В этом пункте исследуем то, какие уникальные значения есть в категориальных столбцах в датасете.

In [34]:
display(data['children'].value_counts())

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце 'children' выявлены возможные опечатки. Для дальнейшего анализа проделаем следующие действия: сроки со значением 20  меняем на значение 2 ( тк в этом случает высокая вероятность опечатки); строки со значением -1 меняем на значение - 0 (тк возможно, эта ошибка связана с различным форматом записи).

In [35]:
# замена значений и вывод уникальных значений после обработки
data['children'] =  data['children'].replace(-1, 0)
data['children'] =  data['children'].replace(20, 2)
display(data['children'].value_counts())

0    14138
1     4808
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

В столбце children некорректных значений не осталось, теперь данные выглядят более правдоподобно.

Далее необходимо разбить столбец с ежемесячным доходом 'total_income' на категории (низкий,средний, высокий) для того, чтобы далее использовать эту информацию при определении как зависит возвращаемость кредита для каждой категории.

In [36]:
# посмотрим еще раз на столбец total_income методом describe и определим медиану методом median
print(data['total_income'].median())
data['total_income'].describe()

142594.39684740017


count    2.145400e+04
mean     1.653201e+05
std      9.818730e+04
min      2.066726e+04
25%      1.076239e+05
50%      1.425944e+05
75%      1.958209e+05
max      2.265604e+06
Name: total_income, dtype: float64

На выходе получили следующие значения дохода : min - 20 667, max - 2 265 604, среднее - 165 320, медиана - 142 594. На основе этих данных распределим клиентов на категории так:
* клиенты с доходом до 80 000 попадают в категорию низкий
* клиенты с доходом от 80 000 до 250 000 попадают в категорию средний
* клиенты с доходом  от 250 000 попадают в категорию высокий

Напищем функцию для категоризации данных столбца с доходами.

In [37]:
# составление функции income_level для распределения клиентов по категориям дохода 
def income_level(total_income):
    if total_income <= 80000:
        return 'низкий доход'
    if 80000 < total_income <= 250000:
        return 'средний доход'
    if total_income > 250000:
        return 'высокий доход'
# применяем функцию к столбцу total_income и сохраняем значения в колонке income_level
data["income_level"] = data['total_income'].apply(income_level)
(data.loc[:,['income_level','total_income']]).head(10)

,income_level,total_income
0,высокий доход,253875.639453
1,средний доход,112080.014102
2,средний доход,145885.952297
3,высокий доход,267628.550329
4,средний доход,158616.077870
5,высокий доход,255763.565419
6,средний доход,240525.971920
7,средний доход,135823.934197
8,средний доход,95856.832424
9,средний доход,144425.938277


**Вывод**

В столбце 'children' были выявлены и исправлены возможные опечатки.

Для упорщения дальнейшего анализа, каждый клиент распределен по категориям, в зависимости от уровня дохода- средний, низкий, высокий доходы.

В остальных столбцах таблицы, категоризация не понадобится, поэтому приступаем к ответу на основные вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на данный вопрос необходимо составить таблицу,  сгруппированную по столбцу children и по ней найти какие из категорий реже и чаще всего возращают кредит (% невозврата). 

In [38]:
# группировка по количеству детей и вычисление количества долгов и % невозврата для каждой категории.
children = data.groupby('children').agg({'debt':['mean','count']})
children.columns = ['% невозврата','кол-во долгов']

children=children.sort_values(by = '% невозврата',ascending = False)
children.style.format({'% невозврата':'{:.2%}'})

,% невозврата,кол-во долгов
children,,
4,9.76%,41
2,9.49%,2128
1,9.23%,4808
3,8.18%,330
0,7.53%,14138
5,0.00%,9


**Вывод**

По таблице children видно, чаще всего не возвращают кредит те, у кого четверо детей (9.76% невозврата) и меньше всего - те у кого детей нет(7.53%). Невозвратов кредита у тех, у кого 5 детей, не зафиксировано.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на данный вопрос необходимо составить таблицу, сгруппированную по столбцу 'family_status' и по ней найти какие из категорий реже и чаще всего возращают кредит (% невозврата).

In [39]:
# группировка по семейному положению и вычисление количества долгов и % невозврата для каждой категории.
status = data.groupby('family_status').agg({'debt':['mean','count']})
status.columns = ['% невозврата','кол-во долгов']
status = status.sort_values(by='% невозврата',ascending = False)
status.style.format({'% невозврата':'{:.2%}'})

,% невозврата,кол-во долгов
family_status,,
Не женат / не замужем,9.75%,2810
гражданский брак,9.35%,4151
женат / замужем,7.55%,12339
в разводе,7.11%,1195
вдовец / вдова,6.57%,959


**Вывод**

По таблице status видно,максимальный процент невозврата кредита зафиксирован среди клиентов,не вступивших в брак (9.75% невозврата среди неженатых / незамужних и 9.35% среди тех,кто состоит в гражданском браке ) и меньше всего - среди вдов/вдовцов (6.57%). 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на данный вопрос необходимо составить таблицу, сгруппированную по столбцу 'income_level' и по ней найти какие из категорий реже и чаще всего возращают кредит (% невозврата).

In [40]:
# группировка по уровню дохода и вычисление количества долгов и % невозврата для каждой категории.
income = data.groupby('income_level').agg({'debt':['mean','count']})
income.columns = ['% невозврата','кол-во долгов']
income = income.sort_values(by = '% невозврата',ascending = False)
income.style.format({'% невозврата':'{:.2%}'})

,% невозврата,кол-во долгов
income_level,,
средний доход,8.39%,16365
низкий доход,7.64%,2276
высокий доход,6.90%,2813


**Вывод**

По таблице income видно, максимальный процент невозврата кредита зафиксирован среди клиентов,со средним доходом (8.39% невозврата) и меньше всего у тех, у кого высокий доход (6.90%).

- Как разные цели кредита влияют на его возврат в срок?

Для ответа на данный вопрос необходимо составить таблицу, сгруппированную по столбцу 'category' и по ней найти какие из категорий реже и чаще всего возращают кредит (% невозврата).

In [41]:
# группировка по цели кредита и вычисление количества долгов и % невозврата для каждой категории.
purpose = data.groupby('category').agg({'debt':['mean','count']})
purpose.columns = ['% невозврата','кол-во долгов']
purpose = purpose.sort_values(by = '% невозврата',ascending = False)
purpose.style.format({'% невозврата':'{:.2%}'})

,% невозврата,кол-во долгов
category,,
автомобиль,9.36%,4306
образование,9.22%,4013
свадьба,8.00%,2324
недвижимость,7.23%,10811


**Вывод**

По таблице purpose видно,что максимальный процент невозврата кредита зафиксирован среди клиентов,которые взяли кредит на покупку автомобиля (9.36% невозврата) и на образование (9.22% невозврата) и минимальный у тех, у кого цель- прибретение недвижимости (6.90%).

## Шаг 4. Общий вывод

Глвной целью исследования было - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

В ходе исследования было выялено, что максимальный процент невозврата кредита зафиксирован среди клиентов,не вступивших в брак (9.75% невозврата среди неженатых / незамужних и 9.35% среди тех,кто состоит в гражданском браке ) и меньше всего - среди вдов/вдовцов (6.57%). Что касается количества детей -  чаще всего не возвращают кредит те, у кого четверо детей (9.76% невозврата) и меньше всего - те у кого детей нет(7.53%).

Разброс в процентах невозрата незначительный(2-3%), поэтому утверждать, что есть явная зависимость между данными факторами и погашением кредита в срок, нельзя.